In [31]:
!pip install requests beautifulsoup4 folium geopy pandas geopandas matplotlib shapely 


Import libraries

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim

In [4]:
# URL of the website to scrape
url = "https://www.clir.org/about-us/current-sponsors-and-funders/"

In [5]:
# Send a GET request to the website and get the HTML content
response = requests.get(url)
html_content = response.content

In [6]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

In [7]:
# Find the unordered list containing the sponsors and members
ul = soup.find("ul", class_="sponsors")

In [16]:
clir = []
dlf = []

In [17]:
for li in ul.find_all("li"):
    text = li.text.strip()
    if "*" in text:
        clir.append(text.replace("*", "").replace("+", ""))
        dlf.append(text.replace("*", "").replace("+", ""))
    elif "+" in text:
        dlf.append(text.replace("*", "").replace("+", ""))
    else:
        clir.append(text)

In [18]:
print("CLIR Sponsors:")
print(clir)
print()
print("DLF Members:")
print(dlf)

CLIR Sponsors:
['Alaska State Library', 'American University', 'Amherst College', 'Arizona State University Libraries', 'Atlanta University Center, Robert W. Woodruff Library', 'Bates College', 'Baylor University', 'Beloit College', 'Berea College', 'Boston College', 'Bowdoin College', 'Brown University Library', 'Bryn Mawr College Libraries', 'California Digital Library', 'Carleton College', 'Carnegie Mellon University', 'Carthage College', 'Casalini Libri', 'The Catholic University of America', 'The Claremont Colleges', 'The Clark Art Institute', 'Coalition for Networked Information', 'Colgate University', 'College of Charleston', 'College of the Holy Cross', 'Columbia University', 'Connecticut College', 'Cornell University Libraries', 'Council of Independent Colleges', 'Dartmouth College', 'Duke University', 'Emory University', 'Florida Atlantic University', 'Furman University', 'George Mason University', 'The George Washington University', 'Georgetown University', 'Georgia Institut

In [19]:
# As a datafram
clir_df = pd.DataFrame({"Sponsor": clir})
dlf_df = pd.DataFrame({"Sponsor": dlf})

clir_df.count()

Sponsor    67
dtype: int64

In [20]:
geolocator = Nominatim(user_agent="clir_geocoder")
location = geolocator.geocode("Washington, DC")
print((location.latitude, location.longitude))
print(location)

(38.8950368, -77.0365427)
Washington, District of Columbia, United States


In [21]:
# rate limit for no timeouts
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [22]:
clir_df['location'] = clir_df['Sponsor'].apply(geocode)
clir_df['point'] = clir_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


RateLimiter caught an error, retrying (0/2 tries). Called with (*('North Carolina State University Libraries',), **{}).
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/homebrew/anaconda3/lib/python3.9/http/client.py", line 1371, in getresponse
    response.begin()
  File "/opt/homebrew/anaconda3/lib/python3.9/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/opt/homebrew/anaconda3/lib/python3.9/http/client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/opt/homebrew/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(

In [23]:
clir_df.head()

,Sponsor,location,point
0,Alaska State Library,"(Alaska State Library, 395, Whittier Street, J...","(58.3001846, -134.415158, 0.0)"
1,American University,"(American University, 4400, Massachusetts Aven...","(38.93804505, -77.08939223651926, 0.0)"
2,Amherst College,"(Amherst College, Norwottuck Rail Trail, Amher...","(42.3703768, -72.5160691823344, 0.0)"
3,Arizona State University Libraries,None,None
4,"Atlanta University Center, Robert W. Woodruff ...","(Robert W Woodruff Library, Beckwith Street So...","(33.7516821, -84.4135688, 0.0)"


In [24]:
dlf_df['location'] = dlf_df['Sponsor'].apply(geocode)
dlf_df['point'] = dlf_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('North Carolina State University Libraries',), **{}).
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/homebrew/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/homebrew/anaconda3/lib/python3.9/http/client.py", line 1371, in getresponse
    response.begin()
  File "/opt/homebrew/anaconda3/lib/python3.9/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/opt/homebrew/anaconda3/lib/python3.9/http/client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/opt/homebrew/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(

In [25]:
dlf_df.head()

,Sponsor,location,point
0,Amherst College,"(Amherst College, Norwottuck Rail Trail, Amher...","(42.3703768, -72.5160691823344, 0.0)"
1,Arizona State University Libraries,None,None
2,"Atlanta University Center, Robert W. Woodruff ...","(Robert W Woodruff Library, Beckwith Street So...","(33.7516821, -84.4135688, 0.0)"
3,Bates College,"(Bates College, Mt. David Trail, Lewiston, And...","(44.1060827, -70.20232209196428, 0.0)"
4,Baylor University,"(Baylor University, 1301, South University Par...","(31.55043405, -97.1102905567766, 0.0)"


In [33]:
import geopandas as gpd

clir_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(clir_df['point'].tolist(), index=clir_df.index)
clir_df["geometry"] = gpd.points_from_xy(clir_df.longitude, clir_df.latitude)

clir_df.head()

,Sponsor,location,point,latitude,longitude,altitude,geometry
0,Alaska State Library,"(Alaska State Library, 395, Whittier Street, J...","(58.3001846, -134.415158, 0.0)",58.300185,-134.415158,0.0,POINT (-134.41516 58.30018)
1,American University,"(American University, 4400, Massachusetts Aven...","(38.93804505, -77.08939223651926, 0.0)",38.938045,-77.089392,0.0,POINT (-77.08939 38.93805)
2,Amherst College,"(Amherst College, Norwottuck Rail Trail, Amher...","(42.3703768, -72.5160691823344, 0.0)",42.370377,-72.516069,0.0,POINT (-72.51607 42.37038)
3,Arizona State University Libraries,None,None,NaN,NaN,NaN,POINT EMPTY
4,"Atlanta University Center, Robert W. Woodruff ...","(Robert W Woodruff Library, Beckwith Street So...","(33.7516821, -84.4135688, 0.0)",33.751682,-84.413569,0.0,POINT (-84.41357 33.75168)


In [34]:
dlf_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(dlf_df['point'].tolist(), index=dlf_df.index)
dlf_df["geometry"] = gpd.points_from_xy(dlf_df.longitude, dlf_df.latitude)

dlf_df.head()


,Sponsor,location,point,latitude,longitude,altitude,geometry
0,Amherst College,"(Amherst College, Norwottuck Rail Trail, Amher...","(42.3703768, -72.5160691823344, 0.0)",42.370377,-72.516069,0.0,POINT (-72.51607 42.37038)
1,Arizona State University Libraries,None,None,NaN,NaN,NaN,POINT EMPTY
2,"Atlanta University Center, Robert W. Woodruff ...","(Robert W Woodruff Library, Beckwith Street So...","(33.7516821, -84.4135688, 0.0)",33.751682,-84.413569,0.0,POINT (-84.41357 33.75168)
3,Bates College,"(Bates College, Mt. David Trail, Lewiston, And...","(44.1060827, -70.20232209196428, 0.0)",44.106083,-70.202322,0.0,POINT (-70.20232 44.10608)
4,Baylor University,"(Baylor University, 1301, South University Par...","(31.55043405, -97.1102905567766, 0.0)",31.550434,-97.110291,0.0,POINT (-97.11029 31.55043)


In [35]:
# Report how many failed
clir_df.latitude.isnull().sum()
clir_df = clir_df[pd.notnull(clir_df['latitude'])]

dlf_df.latitude.isnull().sum()
dlf_df = dlf_df[pd.notnull(dlf_df['latitude'])]


In [56]:
# Convert to geodataframe
from shapely.geometry import Point
points = gpd.GeoSeries(clir_df["geometry"][0]) # todo: fix this
centroid = points.centroid
print(points.centroid)



0    POINT (-134.41516 58.30018)
dtype: geometry


In [96]:
import folium

m = folium.Map(
    location=[37.0902, -95.7129],
    # location=[points.centroid.y, points.centroid.x],
    zoom_start=3,
    tiles="Stamen Toner"
    )

clir_layer = folium.FeatureGroup(name="CLIR Sponsors")

for idx, row in clir_df.iterrows():
    #folium.Marker(location=[row["latitude"], row["longitude"]], tooltip=row["Sponsor"]).add_to(clir_layer)
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]], 
        tooltip=row["Sponsor"], 
        radius=10,
        color="#9C1944",
        fill=True,
        fill_color="#9C1944"
        ).add_to(clir_layer)

clir_layer.add_to(m)

# Create a FeatureGroup for the DLF members
dlf_layer = folium.FeatureGroup(name="DLF Members")

# Add each point to the DLF members layer
for idx, row in dlf_df.iterrows():
    # folium.Marker(location=[row["latitude"], row["longitude"]], tooltip=row["Sponsor"]).add_to(dlf_layer)
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]], 
        tooltip=row["Sponsor"],
        radius=10,
        color="#0067A1",
        fill=True,
        fill_color="#0067A1"
    ).add_to(dlf_layer)

# # Add the DLF members layer to the map
dlf_layer.add_to(m)

# # Add a layer control to the map
folium.LayerControl().add_to(m)

m

In [ ]:
m.save("docs/index.html")